In [1]:
'''
This script shows how to predict stock prices using a basic RNN
'''
import tensorflow as tf
import numpy as np
import matplotlib
import os

tf.set_random_seed(777) # reproducibility

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt


def MinMaxScaler(data):
    ''' Min Max Normalization

    Parameters
    ----------
    data : numpy.ndarray
        input data to be normalized
        shape: [Batch size, dimension]

    Returns
    ----------
    data : numpy.ndarry
        normalized data
        shape: [Batch size, dimension]

    References
    ----------
    .. [1] http://sebastianraschka.com/Articles/2014_about_feature_scaling.html

    '''
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)


# train Parameters
seq_length = 7
data_dim = 5
hidden_dim = 10
output_dim = 5
learning_rate = 0.01
iterations = 500

# Open, High, Low, Volume, Close
xy = np.loadtxt('./data/inputs.csv', delimiter=',')
xy = xy[::-1] # reverse order (chronically ordered)
xy = MinMaxScaler(xy)
x = xy
y = xy[:, [-1]] # Close as label

# build a dataset
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length] # Next close price
    #print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)

# train/test split
train_size = int(len(dataY) * 0.7)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(
    dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(
    dataY[train_size:len(dataY)])

# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None) # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y)) # sum of the squares

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

# RMSE
targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])

rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))


with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))


    inputX = testX[0:1,:,:]
    
    from numpy import newaxis

    testY_hat = []
    
    for i in range(5):
        test_predict = sess.run(Y_pred, feed_dict={X: inputX})
        print test_predict

        inputX = inputX[:,1:,:]
        inputX = np.append(inputX, test_predict[newaxis, :, :], axis=1)
        
        testY_hat = [testY_hat, test_predict];
        



[step: 0] loss: 551.856506348
[step: 1] loss: 402.725799561
[step: 2] loss: 278.913604736
[step: 3] loss: 182.313873291
[step: 4] loss: 117.207847595
[step: 5] loss: 84.9883499146
[step: 6] loss: 81.1049194336
[step: 7] loss: 92.1679153442
[step: 8] loss: 96.8047714233
[step: 9] loss: 88.0680465698
[step: 10] loss: 71.4161148071
[step: 11] loss: 53.8768424988
[step: 12] loss: 40.0232467651
[step: 13] loss: 31.5744132996
[step: 14] loss: 28.1536693573
[step: 15] loss: 28.2720088959
[step: 16] loss: 30.1617546082
[step: 17] loss: 32.2988014221
[step: 18] loss: 33.6187973022
[step: 19] loss: 33.5376205444
[step: 20] loss: 31.8935680389
[step: 21] loss: 28.8724956512
[step: 22] loss: 24.9196300507
[step: 23] loss: 20.621963501
[step: 24] loss: 16.5641555786
[step: 25] loss: 13.1928386688
[step: 26] loss: 10.7368345261
[step: 27] loss: 9.20631599426
[step: 28] loss: 8.45013046265
[step: 29] loss: 8.22612476349
[step: 30] loss: 8.2575712204
[step: 31] loss: 8.28377056122
[step: 32] loss: 8.1

[step: 274] loss: 1.750898242
[step: 275] loss: 1.74439406395
[step: 276] loss: 1.73793935776
[step: 277] loss: 1.73153603077
[step: 278] loss: 1.72517943382
[step: 279] loss: 1.71887457371
[step: 280] loss: 1.71261715889
[step: 281] loss: 1.70640969276
[step: 282] loss: 1.70025193691
[step: 283] loss: 1.69414186478
[step: 284] loss: 1.68808197975
[step: 285] loss: 1.68206882477
[step: 286] loss: 1.67610633373
[step: 287] loss: 1.67019128799
[step: 288] loss: 1.66432368755
[step: 289] loss: 1.65850448608
[step: 290] loss: 1.65273332596
[step: 291] loss: 1.64701008797
[step: 292] loss: 1.64133310318
[step: 293] loss: 1.63570380211
[step: 294] loss: 1.63012135029
[step: 295] loss: 1.62458598614
[step: 296] loss: 1.61909687519
[step: 297] loss: 1.61365377903
[step: 298] loss: 1.60825705528
[step: 299] loss: 1.60290491581
[step: 300] loss: 1.59759879112
[step: 301] loss: 1.59233808517
[step: 302] loss: 1.58712100983
[step: 303] loss: 1.58195030689
[step: 304] loss: 1.57682287693
[step: 305

In [10]:
testY_hat = [testY_hat, test_predict]

In [11]:
testY_hat

[[[[], array([0.67806095], dtype=float32)],
  array([[0.6894759 , 0.69337344, 0.6805838 , 0.67945856, 0.67806095]],
        dtype=float32)],
 array([[0.6894759 , 0.69337344, 0.6805838 , 0.67945856, 0.67806095]],
       dtype=float32)]

In [5]:
    '''
    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    

    # Successful Rate
    sum_score = 0.0;

    for i in range(0, len(test_predict)-1):
        if (test_predict[i,-1] < testY[i+1,-1]) == (test_predict[i,-1] < test_predict[i+1,-1]):
            sum_score = sum_score + 1

    sum_score = sum_score/(len(test_predict)-1)

    print sum_score

    '''
    rmse_val = sess.run(rmse, feed_dict={
                    targets: testY, predictions: test_predict})

    print("RMSE: {}".format(rmse_val))
    '''

    # Plot predictions
    plt.plot(testY)
    plt.plot(test_predict)
    plt.xlabel("Time Period")
    plt.ylabel("Stock Price")
    plt.show()
'''